In [2]:
import datetime
import math
import time
import itertools
from contextlib import closing
from collections import OrderedDict
import json
import numpy as np
import pandas as pd
import MySQLdb
import pymssql
import dfutils

In [3]:
DB_NAME = 'wic'
WIC_DB_HOST = 'wic-risk-database.cwi02trt7ww1.us-east-1.rds.amazonaws.com'
DB_USER = 'aduprey'
DB_PASSWORD = 'aduprey'

In [4]:
def wic_optimized_execute(query, commit=False, retrieve_column_names=False, connection_timeout=250, extra_values=None):
    with closing(MySQLdb.connect(host=WIC_DB_HOST, user=DB_USER, passwd=DB_PASSWORD, db=DB_NAME)) as wic_cnx:
        with closing(wic_cnx.cursor()) as wic_cursor:

            if extra_values is not None:
                wic_cursor.execute(query, (extra_values,))
            else:
                wic_cursor.execute(query)
            if commit:
                wic_cnx.commit()
                return
            fetched_res = wic_cursor.fetchall()  # fetch (and discard) remaining rows

            if retrieve_column_names:
                return fetched_res, [i[0] for i in wic_cursor.description]

            return fetched_res

In [7]:
def get_sleeves_snapshot():
    # region query
    query = "SELECT Fund, Sleeve, InceptionDate, EndDate, " \
            "`Metrics in Bet JSON`,`Metrics in Bet notes JSON`,`Metrics in NAV JSON`,`Metrics in NAV notes JSON` " \
            "FROM " + DB_NAME + ".sleeves_snapshot"
    # endregion

    res = wic_optimized_execute(query)
    cols = ['Fund', 'Sleeve', 'InceptionDate', 'EndDate',
            'Metrics in Bet JSON', 'Metrics in Bet notes JSON', 'Metrics in NAV JSON', 'Metrics in NAV notes JSON']
    return pd.DataFrame(list(res), columns=cols)

In [11]:
def df2row(pivot_col, df):
    dfcols = [c for c in df.columns if c != pivot_col]
    cols = [colname + '|' + period for (colname, period) in itertools.product(dfcols, df[pivot_col])]
    cols2vals = {c:None for c in cols}

    for idx in df.index:
        row = df.loc[idx]
        pivot = row[pivot_col]
        for cln in dfcols:
            key = cln + '|' + pivot
            cols2vals[key] = row[cln]
    res = pd.Series()
    for cln in cols:
        res[cln] = cols2vals[cln]
    return res

def json2row(json):
    df = pd.read_json(json)
    return df2row('Period', df)

In [15]:
def get_sleeves_attribution():
    df = get_sleeves_snapshot()
    metrics_cln = 'Metrics in NAV JSON'

    metrics2include = [('P&L(bps)', 'ITD'), ('P&L($)', 'ITD'),
                           ('P&L(bps)', 'YTD'), ('P&L($)', 'YTD'),
                           ('P&L(bps)', 'QTD'), ('P&L($)', 'QTD'),
                           ('P&L(bps)', 'MTD'), ('P&L($)', 'MTD'),
                           ('P&L(bps)', '5D'), ('P&L($)', '5D'),
                           ('P&L(bps)', '1D'), ('P&L($)', '1D')]

    metric2display_name = {'P&L(bps)': '', 'P&L($)': ''}
    metric2unit = {'P&L(bps)': 'bps', 'P&L($)': '$'}

    fund_codes = ["ARB", "AED", "TACO", "WED", "CAM", "LG", "TAQ","LEV","MACO"]
    sleeve_performance_df = pd.DataFrame()
    for fund_code in fund_codes:
        f_df = df[df['Fund'] == fund_code].copy()
        metrics_df = pd.DataFrame([json2row(json) for json in f_df[metrics_cln]]) # unjsonify metrics, and append columns
        metrics_df.index = f_df.index
        for (metric, period) in metrics2include:
            unit = metric2unit[metric]
            disp_name = metric2display_name[metric]
            display_colname = disp_name + ' ' + period + '(' + unit + ')'
            f_df[display_colname] = metrics_df[metric + '|' + period]

        del f_df['Metrics in NAV JSON'];
        del f_df['Metrics in Bet JSON'];
        del f_df['Metrics in NAV notes JSON'];
        del f_df['Metrics in Bet notes JSON']
        f_df = f_df[(~pd.isnull(f_df[' YTD($)']))]  # don't show null ytds
        f_df.sort_values(by=' YTD($)', inplace=True)

        sleeve_performance_df = sleeve_performance_df.append(f_df)

    return sleeve_performance_df

In [16]:
sleeve_attr_fund_nav = get_sleeves_attribution()
sleeve_attr_fund_nav

,Fund,Sleeve,InceptionDate,EndDate,ITD(bps),ITD($),YTD(bps),YTD($),QTD(bps),QTD($),MTD(bps),MTD($),5D(bps),5D($),1D(bps),1D($)
8,ARB,Credit Opportunities,2016-09-19,2019-04-05,-0.023827,-4.378438e+03,-0.023806,-4.378438e+03,0.330084,5.792359e+04,0.330084,5.792359e+04,0.330084,5.792359e+04,0.190098,33334.993056
9,ARB,Equity Special Situations,2016-01-04,2019-04-05,-0.520436,-9.288939e+04,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
10,ARB,Forwards,2016-01-04,2019-04-05,0.016430,2.937007e+03,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
7,ARB,Break,2016-01-04,2019-04-05,129.652630,2.338678e+07,24.196576,4.285183e+06,0.143253,2.512668e+04,0.143253,2.512668e+04,0.143253,2.512668e+04,0.025751,4515.680917
11,ARB,Merger Arbitrage,2016-01-04,2019-04-05,1339.425563,2.278237e+08,113.766443,2.001712e+07,9.981631,1.750992e+06,9.981631,1.750992e+06,9.981631,1.750992e+06,3.099780,543568.253604
3,AED,Forwards,2016-01-04,2019-04-05,-0.038457,-6.291218e+02,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
0,AED,Break,2016-01-04,2019-04-05,199.519363,2.985046e+06,12.548849,1.627379e+05,0.319757,4.091314e+03,0.319757,4.091314e+03,0.319757,4.091314e+03,0.027239,347.155081
2,AED,Equity Special Situations,2016-01-04,2019-04-05,92.082253,1.703022e+06,44.958301,6.132724e+05,15.815959,2.024841e+05,15.815959,2.024841e+05,15.815959,2.024841e+05,5.532973,70517.715873
1,AED,Credit Opportunities,2016-01-04,2019-04-05,321.506226,4.864575e+06,78.284727,1.040938e+06,-1.268247,-1.585406e+04,-1.268247,-1.585406e+04,-1.268247,-1.585406e+04,1.564891,19944.521061
4,AED,Merger Arbitrage,2016-01-04,2019-04-05,1008.873629,1.481680e+07,115.189748,1.507788e+06,8.086479,1.035098e+05,8.086479,1.035098e+05,8.086479,1.035098e+05,3.457363,44064.069859
